In [ ]:
from sklearn.preprocessing import OneHotEncoder
#from flask import Flask, render_template, request
from sklearn.preprocessing import StandardScaler, MinMaxScaler
#from scipy.stats import zscore
# from sklearn.ensemble import StackingRegressor
from sklearn.neural_network import MLPRegressor
# from sklearn.svm import SVR
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

In [ ]:
def posix_time(dt):
    return (dt - datetime(1970, 1, 1)) / timedelta(seconds=1)

In [ ]:
data = pd.read_csv('/content/traffic_volume_data.csv')

In [ ]:
#print(data['date_time'])
data = data.sort_values(
    by=['date_time'], ascending=True).reset_index(drop=True) #sorting data based on date and time
last_n_hours = [1, 2, 3, 4, 5, 6]
for n in last_n_hours:
    data[f'last_{n}_hour_traffic'] = data['traffic_volume'].shift(n) #creating new colums using traffic_volume
data = data.dropna().reset_index(drop=True)
data.loc[data['is_holiday'] != 'None', 'is_holiday'] = 1 #if holiday then 1 else o
data.loc[data['is_holiday'] == 'None', 'is_holiday'] = 0
data['is_holiday'] = data['is_holiday'].astype(int)

data['date_time'] = pd.to_datetime(data['date_time']) #coverting to date time object
#print(data['date_time'])
#extracting hour,date,month and year from date_time column
data['hour'] = data['date_time'].map(lambda x: int(x.strftime("%H")))
data['month_day'] = data['date_time'].map(lambda x: int(x.strftime("%d")))
data['weekday'] = data['date_time'].map(lambda x: x.weekday()+1)
data['month'] = data['date_time'].map(lambda x: int(x.strftime("%m")))
data['year'] = data['date_time'].map(lambda x: int(x.strftime("%Y")))
data.to_csv("traffic_volume_data.csv", index=None)

In [ ]:
data = pd.read_csv("traffic_volume_data.csv")
data = data.sample(9950,replace=True).reset_index(drop=True)
label_columns = ['weather_type', 'weather_description']
numeric_columns = ['is_holiday', 'temperature',
                       'weekday', 'hour', 'month_day', 'year', 'month']

In [ ]:
features = numeric_columns+label_columns
X = data[features]
#X.head()

In [ ]:
#X.shape

(9950, 9)

In [ ]:
from functools import reduce
def unique(list1):
    ans = reduce(lambda re, x: re+[x] if x not in re else re, list1, [])
    #print(ans)

n1 = data['weather_type']
n2 = data['weather_description']
unique(n1)
unique(n2)
n1features = ['Rain', 'Clouds', 'Clear', 'Snow', 'Mist',
              'Drizzle', 'Haze', 'Thunderstorm', 'Fog', 'Smoke', 'Squall']
n2features = ['light rain', 'few clouds', 'Sky is Clear', 'light snow', 'sky is clear', 'mist', 'broken clouds', 'moderate rain', 'drizzle', 'overcast clouds', 'scattered clouds', 'haze', 'proximity thunderstorm', 'light intensity drizzle', 'heavy snow', 'heavy intensity rain', 'fog', 'heavy intensity drizzle', 'shower snow', 'snow', 'thunderstorm with rain',
              'thunderstorm with heavy rain', 'thunderstorm with light rain', 'proximity thunderstorm with rain', 'thunderstorm with drizzle', 'smoke', 'thunderstorm', 'proximity shower rain', 'very heavy rain', 'proximity thunderstorm with drizzle', 'light rain and snow', 'light intensity shower rain', 'SQUALLS', 'shower drizzle', 'thunderstorm with light drizzle']
"""#Data Preparation"""

n11 = []
n22 = []
for i in range(9950):
    if n1[i] not in n1features:
        n11.append(0)
    else:
        n11.append((n1features.index(n1[i]))+1)
    if n2[i] not in n2features:
        n22.append(0)
    else:
        n22.append((n2features.index(n2[i]))+1)
# print(n11)
# print(n22)
data['weather_type'] = n11
data['weather_description'] = n22

In [ ]:
features = numeric_columns+label_columns
target = ['traffic_volume']
X = data[features]
y = data[target]
#print(y)

In [ ]:
X.head(6)

,is_holiday,temperature,weekday,hour,month_day,year,month,weather_type,weather_description
0,1,277.22,4,0,26,2015,11,7,12
1,1,276.08,1,0,26,2016,12,5,6
2,1,276.08,1,0,26,2016,12,5,6
3,1,274.34,4,0,24,2016,11,5,6
4,1,281.96,5,0,11,2016,11,3,5
5,1,290.08,4,0,4,2013,7,3,5


In [ ]:
x_scaler = MinMaxScaler()
X = x_scaler.fit_transform(X)
y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y).flatten()
warnings.filterwarnings('ignore')

In [ ]:
regr = MLPRegressor(random_state=1, max_iter=500).fit(X, y)
new = []
#print('predicted output :=',regr.predict(X[:10]))
#print('Actual output :=',y[:10])

In [ ]:
# error eval
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
trainX, testX, trainY, testY = train_test_split(X, y, test_size=0.2)
y_pred = regr.predict(testX)
print('Mean Absolute Error:', mean_absolute_error(testY, y_pred))

Mean Absolute Error: 0.07346711053924632


In [ ]:
data[features].head(5)

,is_holiday,temperature,weekday,hour,month_day,year,month,weather_type,weather_description
0,1,277.22,4,0,26,2015,11,7,12
1,1,276.08,1,0,26,2016,12,5,6
2,1,276.08,1,0,26,2016,12,5,6
3,1,274.34,4,0,24,2016,11,5,6
4,1,281.96,5,0,11,2016,11,3,5


In [ ]:
ip1=[]
for i in range(9):
  ip1.append(int(input()))
ip=[ip1]
x_input = x_scaler.transform(ip)
out=regr.predict(x_input)
#print('Before inverse Scaling :',out)

In [ ]:
y_pred = y_scaler.inverse_transform([out])
print('Traffic Volume : ',y_pred)


Traffic Volume :  [[2069.40548048]]


In [ ]:
if(y_pred<=1000):
  print("No Traffic ")
elif y_pred>1000 and y_pred<=3000:
  print("Busy or Normal Traffic")
elif y_pred>3000 and y_pred<=5500:
  print("heavy Traffic")
else:
  print("Worst case")

Busy or Normal Traffic
